In [1]:
import pandas as pd
import numpy as np 
import ast
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

c:\Users\veren\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# read datasets & create dataframes

comdata = pd.read_csv('final_data.csv', index_col= 'Unnamed: 0')
ret_data = pd.read_csv('return_daily.csv', index_col='Unnamed: 0')

In [3]:
market_ret = ret_data[ret_data.index == 'mean']
# market_ret

In [4]:
stock_ret = ret_data.drop('mean', axis=0)
# stock_ret

In [5]:
# comdata.head()

In [6]:
def parse_isin_list(isin_str):
    # Convert string representation of list into actual list
    try:
        isin_list = ast.literal_eval(isin_str)
        # Replace 'None' and actual None with np.nan
        return [np.nan if (item is None or item == 'None') else item for item in isin_list]
    except:
        return []  # Return empty list if parsing fails

In [7]:
comdata['bidders_isin'] = comdata['bidders_isin'].apply(parse_isin_list)

In [8]:
df_exploded = comdata.explode('bidders_isin')
df_exploded['is_bidder'] = df_exploded['bidders_isin'].notna().astype(int)

In [9]:
df_exploded['targets_isin'] = df_exploded['targets_isin'].apply(parse_isin_list)

In [10]:
df_exploded = df_exploded.explode('targets_isin')
df_exploded['is_target'] = df_exploded['targets_isin'].notna().astype(int)

In [11]:
df_exploded.columns

Index(['number', 'date_completion', 'bidders_name', 'date_completion_assumed',
       'date_last_status_update', 'status', 'bidders_industry', 'id',
       'comments', 'targets_industry', 'targets_isin', 'date_rumor',
       'date_postponed', 'date_announcement', 'bidders_isin',
       'date_completion_expected', 'date_withdrawn', 'targets_country', 'type',
       'targets_name', 'bidders_country', 'sentences', 'cleaned_sentences',
       'stemmed_sentences', 'lemmatized_sentences', 'pos_tagged_sentences',
       'dependency_parsed_sentences', 'sentiment', 'vader_sentiment',
       'named_entities', 'topic', 'emotion', 'is_bidder', 'is_target'],
      dtype='object')

In [12]:
senti_cols = ['status', 'bidders_industry', 'bidders_isin', 'targets_industry', 
              'targets_isin', 'date_rumor', 'date_announcement', 'targets_country',
              'bidders_country', 'sentiment', 'vader_sentiment', 'named_entities', 
              'topic', 'emotion', 'is_bidder', 'is_target']

In [13]:
senti_df = df_exploded[senti_cols]

In [14]:
senti_df['ISIN'] = senti_df['bidders_isin'].str.cat(senti_df['targets_isin'], sep='', na_rep='')

C:\Users\veren\AppData\Local\Temp\ipykernel_28868\2310935257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti_df['ISIN'] = senti_df['bidders_isin'].str.cat(senti_df['targets_isin'], sep='', na_rep='')


In [15]:
senti_df.head()

,status,bidders_industry,bidders_isin,targets_industry,targets_isin,date_rumor,date_announcement,targets_country,bidders_country,sentiment,vader_sentiment,named_entities,topic,emotion,is_bidder,is_target,ISIN
0,Completed,['6499'],NaN,['2910'],INE451A01017,2014-07-31,2014-07-31,['IN'],['IN'],0.9118,positive,"[('20140801', 'DATE'), ('310714', 'CARDINAL'),...",3,"[('positive', 0.25)]",0,1,INE451A01017
1,Completed,['2931'],CNE100004C11,['2561'],NaN,2022-04-27,2022-04-27,['CN'],['CN'],0.3612,positive,"[('20220427', 'DATE'), ('automotive electronic...",4,"[('positive', 0.5)]",1,0,CNE100004C11
2,Completed Assumed,['2651'],CNE100000643,['4669'],NaN,2019-09-16,2019-09-16,['CN'],['CN'],0.2960,positive,"[(""'20190916"", 'DATE'), ('beijing', 'GPE'), ('...",1,"[('positive', 0.45)]",1,0,CNE100000643
3,Completed,['2932'],KYG693691092,['2932'],NaN,2015-06-02,2015-06-02,['SG'],['KY'],-0.2960,negative,"[(""'20150602"", 'DATE'), ('5 million', 'CARDINA...",4,"[('positive', 1.0)]",1,0,KYG693691092
4,Withdrawn,['4519'],NaN,['2931'],AU000000SIX0,2019-04-23,2019-04-23,['AU'],['CN'],0.9531,positive,"[('sprintex ltd', 'ORG'), ('11939765', 'DATE')...",3,"[('positive', 0.296875)]",0,1,AU000000SIX0


In [ ]:
# merge dataframes


# e.g.:
# data = sentiment_df.merge(stock_returns_df, on=['ISIN', 'Event_Date'], how='inner')
# data = data.merge(market_returns_df, on='Event_Date', how='inner')

In [ ]:
# Define dependent variable (CAR bzw. CAAR) and independent variables (Sentiment, Market Returns, ...)

X = df[['Sentiment', 'Market_Return']]  # adjust for actual data
y = df['CAR'] # bzw. CAAR

In [ ]:
# split data into training and test datasets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Linear Regression


In [ ]:
# fit regression

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# print coeff and intercept

print(f"Intercept: {model.intercept_}")
print(f"Sentiment Coefficient: {model.coef_[0]}")
print(f"Market Return Coefficient: {model.coef_[1]}")

In [ ]:
# Optionally, evaluate the model on the test set

y_pred = model.predict(X_test)
print(f"Predicted CAR: {y_pred}")
print(f"Actual CAR: {y_test.values}")

In [ ]:
# print R-squared value to check how well the model fits the data

print(f"R-squared: {model.score(X_test, y_test)}")

In [ ]:
print(model.summary())

Ridge Regression

In [ ]:
ridge_regressor = Ridge(alpha=1.0)
ridge_regressor.fit(X_train, y_train)

In [ ]:
print(f"Ridge Intercept: {ridge_regressor.intercept_}")
print(f"Ridge Sentiment Coefficient: {ridge_regressor.coef_[0]}")
print(f"Ridge Market Return Coefficient: {ridge_regressor.coef_[1]}")

In [ ]:
print(ridge_regressor.summary())

Lasso Regression

In [ ]:
lasso_regressor = Lasso(alpha=0.1)
lasso_regressor.fit(X_train, y_train)

In [ ]:
print(f"Lasso Intercept: {lasso_regressor.intercept_}")
print(f"Lasso Sentiment Coefficient: {lasso_regressor.coef_[0]}")
print(f"Lasso Market Return Coefficient: {lasso_regressor.coef_[1]}")

In [ ]:
print(lasso_regressor.summary())

Decision Tree

In [ ]:
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train, y_train)

In [ ]:
print(f"Decision Tree Feature Importances: {dt_regressor.feature_importances_}")

Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

In [ ]:
print(f"Random Forest Feature Importances: {rf_regressor.feature_importances_}")